### **Liskov Substitution**
(SOLID Principles)

In simple terms, it means that objects of a derived (subclass) should be able to replace objects of the base class (parent class) without affecting the correctness of the program. Essentially, if a class S is a subclass of class T, you should be able to use an object of class S wherever an object of class T is expected, without causing any issues or changing the program's behavior.

**Example 1 (Problem)**

In [1]:
class Order:
    items = []
    quantities = []
    prices = []
    status = "open"

    def add_item(self, name, quantity, price):
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)

    def total_price(self):
        total = 0
        for i in range(len(self.prices)):
            total += self.quantities[i] * self.prices[i]
        return total

This is the "Order" class that handles only the order related function.

In [4]:
from abc import ABC, abstractmethod

In [11]:
class PaymentProcessor(ABC):
    @abstractmethod
    def pay(self, order, security_code):
        pass

In [12]:
class DebitPaymentProcessor(PaymentProcessor):
    def pay(self, order, security_code):
        print("Processing debit payment")
        print(f"Verifying security code: {security_code}")
        order.status = "paid"
        print("Payment completed")

class CreditPaymentProcessor(PaymentProcessor):
    def pay(self, order, security_code):
        print("Processing credit payment")
        print(f"Verifying security code: {security_code}")
        order.status = "paid"
        print("Payment completed")

This is the PaymentProcessor class and some of its subclasses for handling the payment.

In [9]:
order = Order()
order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)

print(f'Total price: {order.total_price()}')
payment_handler = DebitPaymentProcessor()
payment_handler.pay(order, "1234567")

Total price: 210
Processing debit payment
Verifying security code: 1234567
Payment completed


In [8]:
class PaypalPaymentProcessor(PaymentProcessor):
    def pay(self, order, security_code):
        print("Processing paypal payment")
        print(f"Verifying email address: {security_code}")
        order.status = "paid"
        print("Payment completed")

The problem arised when we have introduced the paypal payment class. Paypal varifies using email address, not security code. But the "PaymentProcessor" class from which the "PaypalPaymentProcessor" inherited from, has method "pay" which uses the **security code** parameter.

As a result in this scenario, we shouldn't replace instance of "PaymentProcessor" class with instance of "PaypalPaymentProcessor" class.

To solve this issue we can refractor the "PaymentProcessor" class.

**Example 1 (Solution)**

In [13]:
class PaymentProcessor(ABC):
    @abstractmethod
    def pay(self, order):
        pass

We removed the "security code" parameter from the "pay" abstract method

In [18]:
class DebitPaymentProcessor(PaymentProcessor):
    def __init__(self, security_code) -> None:
        super().__init__()
        self.security_code = security_code

    def pay(self, order):
        print("Processing debit payment")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"
        print("Payment completed")


class CreditPaymentProcessor(PaymentProcessor):
    def __init__(self, security_code) -> None:
        super().__init__()
        self.security_code = security_code

    def pay(self, order):
        print("Processing credit payment")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"
        print("Payment completed")


class PaypalPaymentProcessor(PaymentProcessor):
    def __init__(self, email_address) -> None:
        super().__init__()
        self.email_address = email_address

    def pay(self, order):
        print("Processing paypal payment")
        print(f"Verifying email address: {self.email_address}")
        order.status = "paid"
        print("Payment completed")

Then we included the necessary parameter through the initialization function to each of the subclasses.

In [20]:
order = Order()
order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)

print(f'Total price: {order.total_price()}')
payment_handler = PaypalPaymentProcessor("test@gmail.com")
payment_handler.pay(order)

Total price: 840
Processing paypal payment
Verifying email address: test@gmail.com
Payment completed
